# MAPF MaxSAT Solver

Implementación del encoding propuesto en el paper:
**"Multi-Agent Path Finding: A New Boolean Encoding"** (Asín et al., JAIR 2022)

## Estructura del Notebook

1. **Librerías**: Imports necesarios
2. **Encoding**: Función `create_mapf_cnf()` con todas las fórmulas del paper
3. **Método**: Funciones de verificación y resolución con Phase1/Phase2
4. **Test**: Casos de prueba predefinidos
5. **Lector de instancias**: Funciones para cargar y ejecutar instancias desde archivo

### Librerias

In [ ]:
from pysat.formula import WCNF
from pysat.examples.rc2 import RC2
import heapq
from pysat.card import CardEnc, EncType
from pysat.formula import IDPool
from pysat.solvers import Solver
from time import time

### Encoding

In [ ]:
def create_mapf_cnf(agents, grid_size, obstacles, max_time, amo_enc=EncType.ladder, eq_enc=EncType.pairwise):
    wcnf = WCNF()
  
    def dijkstra(u):
        dist = {u: 0.0}
        pq = [(0.0, u)]
        moves = [(1,0), (-1,0), (0,1), (0,-1)]

        while pq:
            d_u, (x, y) = heapq.heappop(pq)
            if d_u > dist[(x, y)]:
                continue
            for dx, dy in moves:
                nx, ny = x + dx, y + dy
                if not (0 <= nx < grid_size[0] and 0 <= ny < grid_size[1]):
                    continue
                if (nx, ny) in obstacles:
                    continue
                nd = d_u + 1  
                if nd < dist.get((nx, ny), float('inf')):
                    dist[(nx, ny)] = nd
                    heapq.heappush(pq, (nd, (nx, ny)))
        return dist
    
    dist_from_start = [dijkstra(agents[a][0]) for a in range(len(agents))]
    dist_to_goal = [dijkstra(agents[a][1]) for a in range(len(agents))]
    
    def feasible(a, t):
        nodes = []
        for x in range(grid_size[0]):
            for y in range(grid_size[1]):
                if (x, y) in dist_from_start[a] and (x, y) in dist_to_goal[a]:
                    if dist_from_start[a][(x, y)] <= t and dist_to_goal[a][(x, y)] <= (max_time - t):
                        nodes.append((x, y))
        return nodes
    
    def neighbors(u):
        x, y = u
        moves = [(0,0),(1,0),(-1,0),(0,1),(0,-1)]
        valid = []
        for dx, dy in moves:
            nx, ny = x+dx, y+dy
            if 0 <= nx < grid_size[0] and 0 <= ny < grid_size[1] and (nx, ny) not in obstacles:
                valid.append((nx, ny))
        return valid

    # Se utiliza PySAT IDPool para compartir el mismo pool de ids entre CardEnc y agentes
    pool = IDPool()
    vp_map = {}

    def _reg(key):
        if key not in vp_map:
            vp_map[key] = pool.id(key)
        return vp_map[key]

    def at(a, u, t):
        x, y = u
        return _reg(('at', a, x, y, t))

    def shift(u, v, t):
        ux, uy = u
        vx, vy = v
        return _reg(('shift', ux, uy, vx, vy, t))

    def finalState(a, t):
        return _reg(('final', a, t))
    
    # Precomputacion de nodos factibles
    feasible_nodes = [[feasible(a, t) for t in range(max_time + 1)] for a in range(len(agents))]

    ## Restricciones duras segun paper (seccion 3.2)

    # C1: cada vertice hace exactamente un shift por paso de tiempo
    # sum sobre v:(u,v) en E de shift(u,v,t) = 1
    for t in range(max_time):
        for x in range(grid_size[0]):
            for y in range(grid_size[1]):
                u = (x, y)
                lits = []
                for v in neighbors(u):
                    lits.append(shift(u, v, t))
                    # H9: evitar shifts opuestos simultaneos -shift(u,v,t) o -shift(v,u,t)
                    # (solo para aristas no-self-loop)
                    if v != u:
                        wcnf.append([-shift(u, v, t), -shift(v, u, t)])
                if lits:
                    res = CardEnc.equals(lits=lits, bound=1, encoding=eq_enc, vpool=pool)
                    wcnf.extend(res.clauses)

    # C2: a lo sumo un agente por vertice en cada tiempo
    # sum sobre a: u en Feasible(a,t) de at(a,u,t) <= 1
    # H10 (follow conflicts): shift(u,v,t) -> shift(v,v,t)
    for t in range(max_time + 1):
        for x in range(grid_size[0]):
            for y in range(grid_size[1]):
                u = (x, y)
                lits = []
                for a in range(len(agents)):
                    if u in feasible_nodes[a][t]:
                        lits.append(at(a, u, t))
                if lits:
                    res = CardEnc.atmost(lits=lits, bound=1, encoding=amo_enc, vpool=pool)
                    wcnf.extend(res.clauses)
                # H10 se aplica solo para t < max_time (shifts van de t a t+1)
                if t < max_time:
                    for v in neighbors(u):
                        wcnf.append([-shift(u, v, t), shift(v, v, t)])

    # Restricciones por agente (H1-H8, H11, C3, S1)
    for a in range(len(agents)):
        # H5: finalState(a, T) - agente esta en estado final al tiempo T
        wcnf.append([finalState(a, max_time)])
        
        # H7: at(a, start(a), 0) - agente comienza en su posicion inicial
        wcnf.append([at(a, agents[a][0], 0)])
        
        # H8: at(a, goal(a), T) - agente termina en su meta al tiempo T
        wcnf.append([at(a, agents[a][1], max_time)])
        
        # Dinamica de movimiento (H1, H2, H4, H11)
        for t in range(max_time):
            for u in feasible_nodes[a][t]:
                neighbors_u = neighbors(u)
                feasible_neighbors = [v for v in neighbors_u if v in feasible_nodes[a][t + 1]]
                
                # H11 (redundante pero util): at(a,u,t) -> (or sobre v en neighbors(u) cap Feasible(a,t+1) de at(a,v,t+1))
                if feasible_neighbors:
                    c = [-at(a, u, t)]
                    c += [at(a, v, t + 1) for v in feasible_neighbors]
                    wcnf.append(c)
                
                # H1 y H2 para cada vecino factible
                for v in feasible_nodes[a][t + 1]:
                    if v in neighbors_u:
                        # H1 (efecto positivo): at(a,u,t) y shift(u,v,t) -> at(a,v,t+1)
                        wcnf.append([-at(a, u, t), -shift(u, v, t), at(a, v, t + 1)])
                        # H2 (explicacion): at(a,u,t) y at(a,v,t+1) -> shift(u,v,t)
                        wcnf.append([-at(a, u, t), -at(a, v, t + 1), shift(u, v, t)])
                
                # H4: no shift hacia vertices no factibles: at(a,u,t) -> -shift(u,v,t) si v no esta en Feasible(a,t+1)
                for v in neighbors_u:
                    if v not in feasible_nodes[a][t + 1]:
                        wcnf.append([-at(a, u, t), -shift(u, v, t)])
        # H6: dinamica de finalState (bicondicional segun paper)
        # at(a, goal(a), t) y -finalState(a, t+1) -> -finalState(a, t)
        # -finalState(a, t) -> at(a, goal(a), t)
        # -finalState(a, t) -> finalState(a, t+1)
        distance = int(dist_from_start[a].get(agents[a][1], float('inf')))
        if distance != float('inf'):
            for t in range(distance, max_time):
                wcnf.append([-at(a, agents[a][1], t), -finalState(a, t + 1), finalState(a, t)])
                wcnf.append([-finalState(a, t), at(a, agents[a][1], t)])
                wcnf.append([-finalState(a, t), finalState(a, t + 1)])
        
        # C3 (redundante pero reemplaza H3): agente en exactamente una ubicacion por tiempo
        # sum sobre u en Feasible(a,t) de at(a,u,t) = 1
        for t in range(max_time + 1):
            lit = [at(a, u, t) for u in feasible_nodes[a][t]]
            if lit:
                res = CardEnc.equals(lits=lit, bound=1, encoding=eq_enc, vpool=pool)
                wcnf.extend(res.clauses)

        # S1 (soft clauses): maximizar tiempo en finalState para minimizar SOC
        # para cada agente a, tiempo t >= d(start(a), goal(a))
        distance = int(dist_from_start[a].get(agents[a][1], float('inf')))
        if distance != float('inf'):
            for t in range(distance, max_time + 1):
                wcnf.append([finalState(a, t)], weight=1)
    try:
        wcnf.nv = pool.top
    except Exception:

        wcnf.nv = max(vp_map.values()) if vp_map else 0
    
    print(f"T = {max_time} | Total vars: {wcnf.nv}, hard clauses: {len(wcnf.hard)}, soft clauses: {len(wcnf.soft)}")
    return wcnf, vp_map, pool


### Método

In [ ]:
def verify_routes(agents, routes, max_time):
        K = len(agents)
        # Verificar conflictos de vertice
        for t in range(max_time + 1):
            occ = {}
            for a in range(K):
                pos = routes.get(a, {}).get(t)
                if pos is None:
                    continue
                if pos in occ:
                    print(f"Vertex conflict at t={t} between agent {a} and {occ[pos]} at {pos}")
                else:
                    occ[pos] = a
        # Verificar conflictos swap y follow
        for t in range(max_time):
            for i in range(K):
                pi = routes.get(i, {}).get(t)
                pi1 = routes.get(i, {}).get(t+1)
                for j in range(i+1, K):
                    pj = routes.get(j, {}).get(t)
                    pj1 = routes.get(j, {}).get(t+1)
                    if pi is None or pj is None:
                        continue
                    if pi1 == pj and pj1 == pi:
                        print(f"Swap conflict between {i} and {j} at time {t}: {pi}->{pi1} and {pj}->{pj1}")
                    if pi1 == pj:
                        print(f"Follow conflict: agent {i} at t+1 equals agent {j} at t (t={t+1})")

def run_case(name, agents, grid_size, obstacles, max_time, amo_enc, eq_enc):
    time_start = time()
    print(f"\n=== CASE: {name} | AMO={amo_enc} EQ={eq_enc} ===")

        # Phase 1: buscar el makespan minimo Tmin tal que las clausulas duras son satisfacibles
    def bfs_shortest(s, g, grid_size, obstacles):
            # BFS para cota inferior del makespan (distancia Manhattan con obstaculos)
        from collections import deque
        q = deque()
        q.append((s, 0))
        seen = {s}
        while q:
            (x, y), d = q.popleft()
            if (x, y) == g:
                return d
            for dx, dy in [(1,0),(-1,0),(0,1),(0,-1)]:
                nx, ny = x+dx, y+dy
                if 0 <= nx < grid_size[0] and 0 <= ny < grid_size[1] and (nx, ny) not in obstacles and (nx, ny) not in seen:
                    seen.add((nx, ny))
                    q.append(((nx, ny), d+1))
        return float('inf')

    lbs = []
    for a in agents:
        lb = bfs_shortest(a[0], a[1], grid_size, obstacles)
        lbs.append(lb if lb != float('inf') else 0)
    lower = max(lbs) if lbs else 0

    def find_min_makespan(min_T, max_T):
        for T in range(min_T, max_T + 1):
            wcnf_T, vp_map_T, pool_T = create_mapf_cnf(agents, grid_size, obstacles, T, amo_enc=amo_enc, eq_enc=eq_enc)
            s = Solver()
            for c in wcnf_T.hard:
                s.add_clause(c)
            ok = s.solve()
            s.delete()
            if ok:
                return T
        return None

    Tmin = find_min_makespan(lower, max_time)
    if Tmin is None:
        print(f"No existe makespan feasible hasta T={max_time} (cota inferior {lower}).")
        elapsed_time = time() - time_start
        # registrar resultado
        try:
            rc2_cost = None
            solved = False
        except Exception as e:
            print('No se pudo escribir resultados:', e)
        return
    print(f"Phase1: Tmin encontrado = {Tmin} (cota inferior {lower})")

    # Phase 2: construir WCNF con Tmin y resolver MaxSAT para optimizar las clausulas blandas
    wcnf, vp_map, pool = create_mapf_cnf(agents, grid_size, obstacles, Tmin, amo_enc=amo_enc, eq_enc=eq_enc)
    rc2 = RC2(wcnf)
    model = rc2.compute()
    if model is None:
        model = getattr(rc2, 'model', None)
    print('Satisfiable:', bool(model), ' coste:', rc2.cost)
    print(f"Time elapsed: {time() - time_start:.2f} seconds")
    if model:
        inv = {v: k for k, v in vp_map.items()}
        true_vars = set(v for v in model if v > 0)
        at_assign = [inv[v] for v in true_vars if v in inv and inv[v][0] == 'at']
        paths = {a: {} for a in range(len(agents))}
        for key in at_assign:
            _, a, x, y, t = key
            paths[a][t] = (x, y)
        for a in range(len(agents)):
            route = [paths[a].get(t, None) for t in range(Tmin + 1)]
            print(f"Agent {a} route:", route)
        verify_routes(agents, paths, Tmin)
    else:
        print('RC2 no devolvio modelo en Phase2. Intentando diagnostico de clausulas duras...')
        try:
            s = Solver()
            selectors = []
            for i, c in enumerate(wcnf.hard):
                sel = pool.id(('sel', i))
                selectors.append(sel)
                s.add_clause(c + [sel])
            ok = s.solve(assumptions=[-s for s in selectors])
            print('Clausulas duras satisfacibles:', ok)
            if not ok:
                core = s.get_core()
                core_idxs = [selectors.index(abs(l)) for l in core if abs(l) in selectors]
                print('Nucleo insat tamaño:', len(core_idxs))
                for idx in core_idxs[:10]:
                    print(f"Clause #{idx}:", wcnf.hard[idx])
            s.delete()
        except Exception as e:
            print('Fallback SAT check failed:', e)

    # Al final, registrar resultados en formato LaTeX
    try:
        elapsed_time = time() - time_start
        rc2_cost = getattr(rc2, 'cost', None)
        solved = bool(model)
    except Exception as e:
        print('No se pudo escribir resultados:', e)


### Test del método

In [ ]:
# Definir tests
'''tests = [
    ("swap-3x3", [((0,0),(2,2)), ((2,0),(0,2))], (3,3), set(), 6),
    ("corridor-1x4", [((0,0),(0,3)), ((0,3),(0,0))], (1,4), set(), 6),
    ("three-agents-3x3", [((0,0),(2,2)), ((2,0),(0,2)), ((0,2),(2,0))], (3,3), set(), 8),
]

    # Opciones de encoding a probar (amo_enc, eq_enc)
enc_options = [
    (EncType.ladder, EncType.pairwise),
    (EncType.pairwise, EncType.pairwise),
]

for name, agents, grid_size, obs, T in tests:
    for amo_enc, eq_enc in enc_options:
        run_case(name, agents, grid_size, obs, T, amo_enc, eq_enc)'''

### Lector de instancias

In [ ]:
def parse_instance_line(line):

    line = line.strip()
    if not line or line.startswith('#'):
        return None
    
    parts = line.split(';')
    if len(parts) != 5:
        print(f"Advertencia: linea mal formada (debe tener 5 partes separadas por ;): {line}")
        return None
    
    try:
        name = parts[0].strip()
        
        # Parsear agentes: [(0,0),(2,2)],[(2,0),(0,2)]
        agents_str = parts[1].strip()
        agents = eval(f"[{agents_str}]")
        
        # Parsear grid: (3,3)
        grid_size = eval(parts[2].strip())
        
        # Parsear obstaculos: [(1,1),(2,2)] o []
        obstacles_str = parts[3].strip()
        obstacles = set(eval(obstacles_str))
        
        # Parsear max_time
        max_time = int(parts[4].strip())
        
        return (name, agents, grid_size, obstacles, max_time)
    except Exception as e:
        print(f"Error parseando linea: {line}")
        print(f"Error: {e}")
        return None


def load_instances_from_file(filename):
    """
    Lee instancias desde un archivo de texto.
    Cada linea debe tener el formato:
    nombre;agentes;grid;obstaculos;max_time
    
    Args:
        filename: ruta al archivo de instancias
    
    Returns:
        lista de tuplas (name, agents, grid_size, obstacles, max_time)
    """
    instances = []
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):
                instance = parse_instance_line(line)
                if instance:
                    instances.append(instance)
        print(f"Cargadas {len(instances)} instancias desde {filename}")
    except FileNotFoundError:
        print(f"Error: No se encontro el archivo {filename}")
    except Exception as e:
        print(f"Error leyendo el archivo: {e}")
    
    return instances


def run_instances_from_file(filename, amo_enc=EncType.ladder, eq_enc=EncType.pairwise):
    """
    Lee y ejecuta instancias desde un archivo.
    
    Args:
        filename: ruta al archivo de instancias
        amo_enc: encoding para at-most-one (default: ladder)
        eq_enc: encoding para equals (default: pairwise)
    """
    instances = load_instances_from_file(filename)
    
    if not instances:
        print("No se encontraron instancias validas para ejecutar")
        return
    
    print(f"\n{'='*60}")
    print(f"Ejecutando {len(instances)} instancias con AMO={amo_enc}, EQ={eq_enc}")
    print(f"{'='*60}\n")
    
    for name, agents, grid_size, obstacles, max_time in instances:
        run_case(name, agents, grid_size, obstacles, max_time, amo_enc, eq_enc)


In [ ]:
run_instances_from_file('instancias.txt', amo_enc=EncType.ladder, eq_enc=EncType.pairwise)

In [ ]:
run_instances_from_file('instancias.txt', amo_enc=EncType.pairwise, eq_enc=EncType.pairwise)